In [ ]:
import json
import pandas as pd
import urllib.request
import urllib.parse

from urllib.parse import urlsplit, quote
from urllib.request import Request, urlopen

from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
url_path_1g = 'https://pokemon.fandom.com/ko/wiki/전국도감/1세대'
url_path_2g = 'https://pokemon.fandom.com/ko/wiki/전국도감/2세대'
url_path_3g = 'https://pokemon.fandom.com/ko/wiki/전국도감/3세대'
url_path_4g = 'https://pokemon.fandom.com/ko/wiki/전국도감/4세대'
url_path_5g = 'https://pokemon.fandom.com/ko/wiki/전국도감/5세대'
url_path_6g = 'https://pokemon.fandom.com/ko/wiki/전국도감/6세대'
url_path_7g = 'https://pokemon.fandom.com/ko/wiki/전국도감/7세대'
url_path_8g = 'https://pokemon.fandom.com/ko/wiki/전국도감/8세대'
url_path_9g = 'https://pokemon.fandom.com/ko/wiki/전국도감/9세대'

In [ ]:
def get_pokedex_list(url_path):
    url_info = urlsplit(url_path)
    encoded_url = f'{url_info.scheme}://{url_info.netloc}{quote(url_info.path)}'
    req = Request(encoded_url, headers={'User-Agent': 'Mozilla/5.0'})
    res = urlopen(req)
    html = res.read()
    soup = BeautifulSoup(html, 'html.parser')
    data = []
    for tr in soup.table.findAll("tr"):
        if tr.th:
            headers = [row.text.strip() for row in tr.findAll("th")]
        else:
            cells = tr.findAll("td")
            img_url = tr.find("img").get("data-src", None)
            if img_url is None:
                img_url = tr.find("img")['src']
            datum = [row.text.strip() for row in cells]
            types = [types.text for types in cells[-1].findAll("a")]
            datum[2] = img_url
            datum[-1] = "+".join(types)
            data.append(datum)
    df = pd.DataFrame(data, columns=headers)
    df.drop(df.columns[0], axis=1, inplace=True)
    return df

In [ ]:
get_pokedex_list(url_path_4g)

In [ ]:
def get_pokedex_to_json(target_number=1017):
    url_path = urlsplit('https://pokemon.fandom.com/ko/wiki/이상해씨_(포켓몬)')
    encoded_url = f'{url_path.scheme}://{url_path.netloc}{quote(url_path.path)}'
    info = []
    cnt = 0
    for _ in tqdm(range(target_number + 2)):
        cnt += 1
        req = Request(encoded_url, headers={'User-Agent': 'Mozilla/5.0'})
        res = urlopen(req)
        html = res.read()
        soup = BeautifulSoup(html, 'html.parser')

        name = soup.find("div", {"class": "name-ko"}).text.strip()
        number = soup.find("div", {"class": "index"}).text.strip()
        try:
            img_url = soup.find("div", {"class": "image rounded"}).find("a", {"class": "image"})["href"]
            filepath = f"images/{number.replace('.', '_')}_{name}.png"
            urllib.request.urlretrieve(img_url, filepath)
        except:
            filepath = None
        doc_text = '\n'.join([p.text.replace('\n', '').strip() for p in soup.find_all("p")])
        types = [poke_type['title'].split(' ')[0].strip() for poke_type in
                 soup.select('tbody > tr > td > div')[0].select('span > a')]

        info.append(dict(
            name=name,
            number=number,
            types=types,
            doc_text=doc_text,
            image_path=filepath,
            url=encoded_url
        ))
        next_monster = soup.find("table").findAll("a")[-1]['href']
        encoded_url = "https://pokemon.fandom.com" + next_monster
        if number == f"No.{target_number:04d}":
            break
        if cnt >= target_number:
            break

    pd.DataFrame(info).to_csv('pokemon.csv', index=False)
    with open('pokemon.json', 'w') as f:
        json.dump(info, f, ensure_ascii=False, indent=4)

In [ ]:
get_pokedex_to_json(1)

In [ ]:
import cairosvg

svg_code = '''
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" viewBox="0 0 18 18" width="1em" height="1em">
    <defs>
        <path id="IconImagesSmall__a" d="M11 15h-.586L3 7.586V7h8v8zm-8 0v-4.586L7.586 15H3zm9-10a1 1 0 011 1v10a1 1 0 01-1 1H2a1 1 0 01-1-1V6a1 1 0 011-1h10zm4-4a1 1 0 011 1v10a1 1 0 11-2 0V3H6a1 1 0 110-2h10z"></path>
    </defs>
    <use xlink:href="#IconImagesSmall__a" fill-rule="evenodd"></use>
</svg>
'''

# SVG를 PNG로 변환하고 파일로 저장


print("SVG가 PNG 파일로 성공적으로 저장되었습니다.")